In [1]:
! pip3 install PyPDF2 transformers torch chromadb sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tomli-2.2.1-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 30.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 10.4 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 11.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 14.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 9.0 MB/s eta 0:00:00:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
import chromadb
import PyPDF2
! pip install groq
from groq import Groq
from typing import List
from sentence_transformers import SentenceTransformer

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


/home/dhairya/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class PDFRAGSystem:
    def __init__(self,
                 embedding_model='all-MiniLM-L6-v2',
                 groq_model='mixtral-8x7b-32768',
                 api_key=None):
        """
        Initialize the PDF-based RAG system for Colab

        Args:
            embedding_model (str): Hugging Face embedding model
            groq_model (str): Groq LLM model to use
            api_key (str): Groq API Key
        """
        # Initialize Groq Client
        from google.colab import userdata
        from getpass import getpass

        if api_key is None:
            try:
                # Option 1: Using Colab Secrets
                api_key = userdata.get('GROQ_API_KEY')
            except:
                # Option 2: Manual input
                api_key = getpass("Enter your Groq API Key: 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 ")

        self.groq_client = Groq(api_key=api_key)

        # Initialize Embedding Model
        self.embedding_model = SentenceTransformer(embedding_model)

        # Initialize ChromaDB Client with persistent storage
        self.chroma_client = chromadb.PersistentClient(path="./chroma_db")

        # Groq LLM Model
        self.groq_model = groq_model

    def upload_pdf(self, pdf_path: str) -> List[str]:
        """
        Upload PDF and extract text

        Args:
            pdf_path (str): Path to the PDF file

        Returns:
            List of text chunks from PDF
        """
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text_chunks = []

            for page in pdf_reader.pages:
                # Split text into chunks (e.g., 500 characters)
                page_text = page.extract_text()
                chunks = [
                    page_text[i:i+500]
                    for i in range(0, len(page_text), 500)
                ]
                text_chunks.extend(chunks)

        return text_chunks

    def create_embeddings(self, text_chunks: List[str]) -> List[List[float]]:
        """
        Convert text chunks to embeddings

        Args:
            text_chunks (List[str]): List of text chunks

        Returns:
            List of embeddings
        """
        return [
            self.embedding_model.encode(chunk).tolist()
            for chunk in text_chunks
        ]

    def save_to_chromadb(self,
                          text_chunks: List[str],
                          embeddings: List[List[float]],
                          collection_name: str = None):
        """
        Save text chunks and embeddings to ChromaDB

        Args:
            text_chunks (List[str]): Text chunks
            embeddings (List[List[float]]): Embeddings for text chunks
            collection_name (str): Name of ChromaDB collection

        Returns:
            ChromaDB collection
        """
        # Generate a unique collection name if not provided
        if collection_name is None:
            collection_name = f"pdf_collection_{len(text_chunks)}"

        # Delete existing collection if it exists
        try:
            self.chroma_client.delete_collection(name=collection_name)
        except:
            pass

        # Create new collection
        collection = self.chroma_client.create_collection(name=collection_name)

        # Add chunks and embeddings
        for idx, (chunk, embedding) in enumerate(zip(text_chunks, embeddings)):
            collection.add(
                ids=[str(idx)],
                embeddings=embedding,
                documents=[chunk]
            )

        return collection

    def retrieve_context(self,
                         query: str,
                         collection,
                         top_k: int = 3):
        """
        Retrieve relevant context for a query

        Args:
            query (str): User query
            collection: ChromaDB collection
            top_k (int): Number of top results to retrieve

        Returns:
            List of most relevant text chunks
        """
        query_embedding = self.embedding_model.encode(query).tolist()

        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k
        )

        return results['documents'][0]

    def generate_response(self,
                          query: str,
                          context: List[str]) -> str:
        """
        Generate response using Groq LLM

        Args:
            query (str): User query
            context (List[str]): Retrieved context

        Returns:
            Generated response
        """
        # Construct context-aware prompt
        context_str = "\n".join(context)
        prompt = f"""
        Context: {context_str}

        Question: {query}

        Based on the context, provide a detailed and accurate answer.
        """

        chat_completion = self.groq_client.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": prompt}
            ],
            model=self.groq_model
        )

        return chat_completion.choices[0].message.content

In [4]:
# Colab Usage Example
def main():
    # Option 1: Manually upload PDF in Colab
    from google.colab import files
    print("Please upload your PDF file:")
    uploaded = files.upload()
    pdf_path = list(uploaded.keys())[0]  # Get the first uploaded file

    # Initialize RAG System
    rag_system = PDFRAGSystem()

    # Step 1: Upload PDF
    text_chunks = rag_system.upload_pdf(pdf_path)

    # Step 2: Create Embeddings
    embeddings = rag_system.create_embeddings(text_chunks)

    # Step 3: Save to ChromaDB (with automatic unique collection name)
    collection = rag_system.save_to_chromadb(text_chunks, embeddings)

    # Step 4: Interactive Retrieval and Generation
    while True:
        query = input("Ask a question (or type 'exit'): ")
        if query.lower() == 'exit':
            break

        # Retrieve context
        context = rag_system.retrieve_context(query, collection)

        # Generate response
        response = rag_system.generate_response(query, context)
        print("Response:", response)

# Uncomment to run
main()

ModuleNotFoundError: No module named 'google.colab'